In [2]:
import os
import gc
import psutil
import timeit
import trimesh
import pyrender
import numpy as np

In [3]:
def load_step_part(filepath):
    mesh = trimesh.load_mesh(filepath)
    mesh.apply_translation(-mesh.centroid)
    mesh.apply_scale(1000)
    return mesh


def path_to_mesh(path):
    lines = []
    for entity in path.entities:
        segment = entity.discrete(path.vertices)
        for i in range(len(segment) - 1):
            lines.append(segment[i])
            lines.append(segment[i + 1])

    lines_np = np.array(lines, dtype=np.float32)

    line_mesh = pyrender.Primitive(positions=lines_np, mode=1, material=pyrender.MetallicRoughnessMaterial(baseColorFactor=(0, 0, 0)))
    return pyrender.Mesh([line_mesh])

In [4]:
MODEL_DIR = "models/"
IMG_SIZE = 512

step_files = [f for f in os.listdir(MODEL_DIR) if f.lower().endswith(".step")]
r = pyrender.OffscreenRenderer(512, 512)

In [20]:
process = psutil.Process(os.getpid())
start_time = timeit.default_timer()

meshes = []
outline_meshes = []
peak_memory_usage = -1
t_mesh_load_time = 0
t_outline_load_time = 0
t_render_time = 0
last_memory_usage = 0

for i, f in enumerate(step_files[353:]):
    try:
        i_start_time = timeit.default_timer()
        scene = pyrender.Scene(bg_color=(0.95, 0.95, 0.95, 1.0), ambient_light=(0.5, 0.5, 0.5, 1.0))
        scene.add(pyrender.PerspectiveCamera(yfov = np.pi / 3.0))
        mesh = load_step_part(os.path.join(MODEL_DIR, f))
        meshes.append(mesh)
        mesh_load_time = timeit.default_timer() - i_start_time
        t_mesh_load_time += mesh_load_time

        i2_start_time = timeit.default_timer()
        m = mesh.copy()
        m.merge_vertices()
        outline = path_to_mesh(
            trimesh.load_path(
                m.vertices[
                    m.face_adjacency_edges[
                        m.face_adjacency_angles >= np.radians(90 - max(0, min((200000 - len(m.vertices)) * 0.0005, 80)))
                    ]
                ]
            )
        )
        outline_meshes.append(outline)
        outline_load_time = timeit.default_timer() - i2_start_time
        t_outline_load_time += outline_load_time

        i3_start_time = timeit.default_timer()
        scene.add(pyrender.Mesh.from_trimesh(mesh))
        scene.add(outline)
        for _ in range(6):
            r.render(scene, flags=pyrender.RenderFlags.SKIP_CULL_FACES)
        render_time = timeit.default_timer() - i3_start_time
        t_render_time += render_time

        memory_usage = process.memory_info().rss / (1024 * 1024)
        peak_memory_usage = max(memory_usage, peak_memory_usage)

        del m
        gc.collect()
        print(f"[{i}/{len(step_files)}] Loaded {os.path.splitext(f)[0]} in {(timeit.default_timer() - i_start_time):.2f}s ({mesh_load_time:.2f} mesh / {outline_load_time:.2f} outline / {render_time:.2f} render). Current memory usage is {memory_usage:.2f} MiB, {((memory_usage - last_memory_usage) * 1024):.2f} KiB delta.")
        last_memory_usage = memory_usage
    except:
        print(f"[{i}/{len(step_files)}] Failed to load {os.path.splitext(f)[0]}")

print(f"Loaded all components in {(timeit.default_timer() - start_time):.2f}s ({t_mesh_load_time:.2f} mesh / {t_outline_load_time:.2f} outline / {t_render_time:.2f} render). Peak memory usage was {peak_memory_usage} MiB.")

[0/733] Loaded OpenGoRAIL48 in 7.09s (5.54 mesh / 1.04 outline / 0.36 render). Current memory usage is 2929.25 MiB, 2999552.00 KiB delta.
[1/733] Loaded OpenGoRAIL528 in 7.09s (5.57 mesh / 1.04 outline / 0.34 render). Current memory usage is 1528.61 MiB, -1434260.00 KiB delta.
[2/733] Loaded OpenGoRAIL624 in 7.07s (5.56 mesh / 1.02 outline / 0.35 render). Current memory usage is 1691.87 MiB, 167184.00 KiB delta.
[3/733] Loaded OpenGoRAIL72 in 7.09s (5.61 mesh / 1.02 outline / 0.36 render). Current memory usage is 1851.14 MiB, 163092.00 KiB delta.
[4/733] Loaded OpenGoRAIL912 in 7.14s (5.65 mesh / 1.01 outline / 0.36 render). Current memory usage is 2014.53 MiB, 167308.00 KiB delta.
[5/733] Loaded OpenGoRAIL96 in 7.24s (5.73 mesh / 1.02 outline / 0.34 render). Current memory usage is 2177.03 MiB, 166400.00 KiB delta.
[6/733] Loaded PaddleTire in 6.98s (4.58 mesh / 1.97 outline / 0.31 render). Current memory usage is 2251.26 MiB, 76016.00 KiB delta.
[7/733] Loaded PatternPlate1x1H in 0.2

In [20]:
m = load_step_part(os.path.join(MODEL_DIR, step_files[357]))

In [21]:
path = trimesh.load_path(
    m.vertices[
        m.face_adjacency_edges[
            m.face_adjacency_angles
            >= np.radians(90 - max(0, min((200000 - len(m.vertices)) * 0.0005, 80)))
        ]
    ]
)

In [23]:
m_path = path_to_mesh(path)